# Creating a Dataset for American Movies (1970-2023) from Scratch

In this notebook, we undertake the task of creating a comprehensive dataset for American movies released between 1970 and 2024. The process involves several key stages:

## 1. Data Collection

### 1.1 Retrieving Movie Titles, Cast and Crew, and Release Year from American Movies

In this stage, we dynamically construct Wikipedia URLs for each year from 1970 to 2024 to gather essential information such as movie titles, cast and crew details, and release years.

## 2. Data Ingestion

### 2.1 Loading Data into the Notebook Environment

Once the necessary data is collected, we proceed to ingest it into the notebook environment. This step involves reading the data The Movie Data Base (TMDb) API.

## 3. Data Processing

### 3.1 Cleaning and Preprocessing the Data

With the data loaded into the notebook, we perform essential preprocessing tasks such as duplicate removal, missing value imputation, and text cleaning. These steps ensure the data's quality and readiness for further analysis.

## 4. Data Cleaning

### 4.1 Ensuring Data Quality and Consistency

In this final stage, we conduct quality assurance checks to verify the integrity and consistency of the dataset. This involves validating data ranges, performing consistency checks, and implementing error handling mechanisms.



In [1]:
import os
import requests
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
from dotenv import load_dotenv

In [2]:
# We construct dynamically Wikipedia URLs for each year from 1970 to 2023.
def get_films(year):

    url = f"https://en.wikipedia.org/wiki/List_of_American_films_of_{year}"
    df = pd.read_html(url, header=0)

    # Take relevant tables only
    df = pd.concat(df[2:6])
    df.columns = df.columns.str.lower()
    if 'ref.' in df.columns:
        df = df.drop(['opening', 'opening.1', 'production company', 'ref.'], axis=1)
    else:
        df = df.drop(['opening', 'opening.1', 'production company'], axis=1)
    df['year'] = year
    return df

In [ ]:
df = pd.DataFrame()
for year in range(1970,2017):
    print(year)
    df = pd.concat([df, get_films(year)])

In [4]:
df.head()

,title,cast and crew,year,unnamed: 5,"$217,581,231"
0,Scream and Scream Again,Gordon Hessler (director); Christopher Wicking...,1970,NaN,NaN
1,Jenny,George Bloomfield (director/screenplay); Marti...,1970,NaN,NaN
2,The Adventures of Gerard,Jerzy Skolimowski (director/screenplay); Arthu...,1970,NaN,NaN
3,...tick...tick...tick...,Ralph Nelson (director); James Lee Barrett (sc...,1970,NaN,NaN
4,Last of the Mobile Hot Shots,Sidney Lumet (director); Gore Vidal (screenpla...,1970,NaN,NaN


In [5]:
df = df.drop(labels=['unnamed: 5', '$217,581,231'], axis=1)
df

,title,cast and crew,year
0,Scream and Scream Again,Gordon Hessler (director); Christopher Wicking...,1970
1,Jenny,George Bloomfield (director/screenplay); Marti...,1970
2,The Adventures of Gerard,Jerzy Skolimowski (director/screenplay); Arthu...,1970
3,...tick...tick...tick...,Ralph Nelson (director); James Lee Barrett (sc...,1970
4,Last of the Mobile Hot Shots,Sidney Lumet (director); Gore Vidal (screenpla...,1970
...,...,...,...
41,Silence,Martin Scorsese (director/screenplay); Jay Coc...,2016
42,Why Him?,John Hamburg (director/screenplay); Ian Helfer...,2016
43,Hidden Figures,Theodore Melfi (director/screenplay); Allison ...,2016
44,Live by Night,Ben Affleck (director/screenplay); Ben Affleck...,2016


For 2017 there is a typo in wikitable class, so let's processing separately.

In [7]:
from bs4 import BeautifulSoup
from io import StringIO
import requests

url = "https://en.wikipedia.org/wiki/List_of_American_films_of_2017"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find the problematic cell and set the rowspan attribute to '1'
problematic_cell = soup.find('td', {'style': 'text-align:center; background:#dfc7f7', 'rowspan': 'q'})
if problematic_cell:
    problematic_cell['rowspan'] = '1'

# Get the updated HTML content
updated_html = str(soup)

# Read the HTML table into a DataFrame
df_list = pd.read_html(StringIO(updated_html))
df_list = pd.concat(df_list[2:6])
df_list.columns = df_list.columns.str.lower()
df_list = df_list.drop(['opening', 'opening.1', 'production company', 'ref.'], axis=1)
df_list['year'] = 2017
df_list

,title,cast and crew,year
0,Underworld: Blood Wars,Anna Foerster (director); Cory Goodman (screen...,2017
1,Arsenal,Steven C. Miller (director); Jason Mosberg (sc...,2017
2,Between Us,Rafael Palacio Illingworth (director/screenpla...,2017
3,Monster Trucks,Chris Wedge (director); Derek Connolly (screen...,2017
4,The Bye Bye Man,Stacy Title (director); Jonathan Penner (scree...,2017
...,...,...,...
56,Crooked House,Gilles Paquet-Brenner (director/screenplay); J...,2017
57,Hostiles,Scott Cooper (director/screenplay); Christian ...,2017
58,All the Money in the World,Ridley Scott (director); David Scarpa (screenp...,2017
59,Molly's Game,Aaron Sorkin (director/screenplay); Jessica Ch...,2017


In [8]:
df = pd.concat([df, df_list])
df

,title,cast and crew,year
0,Scream and Scream Again,Gordon Hessler (director); Christopher Wicking...,1970
1,Jenny,George Bloomfield (director/screenplay); Marti...,1970
2,The Adventures of Gerard,Jerzy Skolimowski (director/screenplay); Arthu...,1970
3,...tick...tick...tick...,Ralph Nelson (director); James Lee Barrett (sc...,1970
4,Last of the Mobile Hot Shots,Sidney Lumet (director); Gore Vidal (screenpla...,1970
...,...,...,...
56,Crooked House,Gilles Paquet-Brenner (director/screenplay); J...,2017
57,Hostiles,Scott Cooper (director/screenplay); Christian ...,2017
58,All the Money in the World,Ridley Scott (director); David Scarpa (screenp...,2017
59,Molly's Game,Aaron Sorkin (director/screenplay); Jessica Ch...,2017


Continue up to 2023

In [12]:
for year in range(2018,2024):
    print(year)
    df = pd.concat([df, get_films(year)])

2018
2019
2020
2021
2022
2023


In [13]:
df

,title,cast and crew,year
0,Scream and Scream Again,Gordon Hessler (director); Christopher Wicking...,1970
1,Jenny,George Bloomfield (director/screenplay); Marti...,1970
2,The Adventures of Gerard,Jerzy Skolimowski (director/screenplay); Arthu...,1970
3,...tick...tick...tick...,Ralph Nelson (director); James Lee Barrett (sc...,1970
4,Last of the Mobile Hot Shots,Sidney Lumet (director); Gore Vidal (screenpla...,1970
...,...,...,...
100,Memory,Michel Franco (director/screenplay); Jessica C...,2023
101,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",2023
102,The Boys in the Boat,"George Clooney (director), Mark L. Smith (scre...",2023
103,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023


Before saving to a file, let's be sure that there are no null values in the title and year fields.

In [14]:
df.isna().sum()

title            102
cast and crew    108
year               0
dtype: int64

In [15]:
df.dropna(subset=['title'], inplace=True)

In [18]:
len(df)

11890

In [19]:
df.reset_index(drop=True, inplace=True)
df

,title,cast and crew,year
0,Scream and Scream Again,Gordon Hessler (director); Christopher Wicking...,1970
1,Jenny,George Bloomfield (director/screenplay); Marti...,1970
2,The Adventures of Gerard,Jerzy Skolimowski (director/screenplay); Arthu...,1970
3,...tick...tick...tick...,Ralph Nelson (director); James Lee Barrett (sc...,1970
4,Last of the Mobile Hot Shots,Sidney Lumet (director); Gore Vidal (screenpla...,1970
...,...,...,...
11885,Memory,Michel Franco (director/screenplay); Jessica C...,2023
11886,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",2023
11887,The Boys in the Boat,"George Clooney (director), Mark L. Smith (scre...",2023
11888,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023


In [26]:
df.to_csv('./data/movies_1970-2023.csv')

## 2. Data Ingestion. Retrieving movie IDs from "The Movie Data Base"

In the second phase of this project, we shifted our focus to Data Ingestion, specifically aimed at retrieving unique movie identifiers (IDs) from "The Movie Database" (TMDb). This step was crucial for augmenting our dataset with additional information and establishing a comprehensive linkage between movies. By interfacing with TMDb's API, we accessed movie IDs, which serve as universal identifiers across various platforms. This integration not only enriched our dataset but also opened avenues for obtaining detailed metadata and ratings, setting the stage for a more robust and insightful analysis of movies.

In [28]:
import numpy as np

def parsing_fun(row, data_json):
    # Initialize an empty list to store data for matching movies
    matched_movies_data = []

    # Check if 'total_results' list is not empty
    if data_json.get('total_results'):
        # Iterate through the 'results' list in the JSON data
        for movie in data_json.get('results'):
            # Extract the release date
            release_date = movie.get('release_date', '')

            # Check if there is a release data
            if release_date == '':
                release_date = '0'

            # Check if the movie is from the specified year and has a title similar to the provided title
            delta = np.abs(int(release_date.split('-')[0]) - row.get('year'))
            if release_date and (delta <= 1):
                matched_movies_data.append({
                    "id": int(movie.get('id', np.nan)),
                    "genre_ids": movie.get('genre_ids', np.nan),
                    "original_title": movie.get('original_title', ''),
                    "title": row.get('title', ''),
                    "cast and crew": row.get('cast and crew', ''),
                    "year": row.get('year', np.nan),
                    "poster_path": movie.get('poster_path', ''),
                })

    else:
        # Append NaN values if 'results' list is empty
        matched_movies_data.append({
            "id": np.nan,
            "genre_ids": np.nan,
            "original_title": '',
            "title": row.get('title', ''),
            "cast and crew": row.get('cast and crew', ''),
            "year": row.get('year', np.nan),
            "poster_path": '',
        })

    # Convert the list of dictionaries to a DataFrame
    matched_movies_df = pd.DataFrame(matched_movies_data)

    return matched_movies_df

In [ ]:
load_dotenv()

my_api_key = os.environ.get('tmdb_api_key')

# Create an empty DataFrame with the desired columns
header_df = pd.DataFrame(columns=['id', 'genre_ids', 'original_title', 'title', 'cast and crew', 'year', 'poster_path'])

FILE_NAME = './data/movie_ids_.csv'
# Save the header DataFrame to a CSV file
header_df.to_csv(FILE_NAME, index=False)

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):

    title = row['title']
    year = row['year']
    
    while True:
        try:
            response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key={my_api_key}&query={title}")
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
            
            if response.status_code == 200:
                data_json = response.json()

                current_data = parsing_fun(row, data_json)
                
                # Append to the list
                current_data.to_csv(FILE_NAME, mode='a', header=False, index=False)
                
                break  # Exit the loop if the request is successful
            
        except requests.exceptions.RequestException as e:
            print(f"Error for {title}: {e}")
            # Handle specific exceptions if needed
        
        if response.status_code == 429:
            print(f"Rate limited. Waiting for 10 seconds before retrying...")
            time.sleep(10)
        else:
            break  # Exit the loop if it's not a rate-limiting issue

    time.sleep(0.3)  # Optional: Add a small delay to avoid rate limiting

## 3. Data Processing: Step 1 - Deleting Empty Fields

Given that the movie was not found in the TMDb database, let's proceed by deleting any row where the first three fields are empty. Most of these entries originate from countries other than the USA.

```
,,,Carry On Up the Jungle (United Kingdom),"Gerald Thomas"
```

In [32]:
df = pd.read_csv('./data/movies_id.csv')
df

,id,genre_ids,original_title,title,cast_and_crew,year,poster_path
0,42594.0,"[27, 878]",Scream and Scream Again,Scream and Scream Again,Gordon Hessler (director); Christopher Wicking...,1970,/49m9QJ2ubKuVtdDj9B7XqgCyriv.jpg
1,280133.0,"[18, 10749]",Jenny,Jenny,George Bloomfield (director/screenplay); Marti...,1970,/iaJ0PyRh17KoMcUlXxTVu6Vi85s.jpg
2,225155.0,"[12, 35, 18]",The Adventures of Gerard,The Adventures of Gerard,Jerzy Skolimowski (director/screenplay); Arthu...,1970,/aoOV7vWvGiNjaUTHCSIEAeHSvOw.jpg
3,85255.0,"[18, 28]",… tick… tick… tick…,...tick...tick...tick...,Ralph Nelson (director); James Lee Barrett (sc...,1970,/rBNgytHXZEXgOl805pYWKiBQe7s.jpg
4,117999.0,[18],Last of the Mobile Hot Shots,Last of the Mobile Hot Shots,Sidney Lumet (director); Gore Vidal (screenpla...,1970,/xj5Rub6H5B9yMh6GL3oomOM1SLg.jpg
...,...,...,...,...,...,...,...
16986,365620.0,"[18, 36]",Ferrari,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,/nNMoJMDCeF4Q5wpWvKuh5b8K2sX.jpg
16987,959286.0,[16],Бизнес войны. Феррари против Ламборгини,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,NaN
16988,1036619.0,"[35, 18, 10749]",Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/wJFHUglmh7xlY7g9Ql85QAJBZ6n.jpg
16989,1065505.0,[18],Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/4o5qpndKjkLTPHqBrmg6v0W3m13.jpg


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16991 entries, 0 to 16990
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              16761 non-null  float64
 1   genre_ids       16761 non-null  object 
 2   original_title  16761 non-null  object 
 3   title           16991 non-null  object 
 4   cast_and_crew   16984 non-null  object 
 5   year            16991 non-null  int64  
 6   poster_path     16086 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 929.3+ KB


In [34]:
df.dropna(subset='id', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16761 entries, 0 to 16990
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              16761 non-null  float64
 1   genre_ids       16761 non-null  object 
 2   original_title  16761 non-null  object 
 3   title           16761 non-null  object 
 4   cast_and_crew   16758 non-null  object 
 5   year            16761 non-null  int64  
 6   poster_path     16086 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.0+ MB


Let's manually retrieve the cast and crew information for three movies from their respective Wikipedia pages. Below are the details obtained for each movie:

In [36]:
df[df.cast_and_crew.isna()]

,id,genre_ids,original_title,title,cast_and_crew,year,poster_path
205,198652.0,"[35, 80]",Loot,Loot,NaN,1970,/iMOaKS5IEKtazHuELLfzd0rXGYo.jpg
2306,190544.0,"[878, 53, 18, 14]",Image of the Beast,Image of the Beast,NaN,1981,/bFeYX1CJjovUBr7vQXKa5aW9F3X.jpg
10526,44773.0,"[12, 28, 14]",The 7 Adventures of Sinbad,The 7 Adventures of Sinbad,NaN,2010,/c6jY8DaiumXg02CMUdcdwm7thXy.jpg


In [37]:
df.loc[205, 'cast_and_crew'] = "Silvio Narizzano (director); Ray Galton and Alan Simpson (screenplay); 	Richard Attenborough, Lee Remick, Hywel Bennett, Milo O'Shea, Roy Holder"

In [38]:
df.loc[2306, 'cast_and_crew'] = "Donald W. Thompson (director); Donald W. Thompson, Russell S. Doughten Jr. (story/screenplay); William Wellman Jr., Susan Plumb, Patty Dunning"

In [39]:
df.loc[10526, 'cast_and_crew'] = "Adam Silver, Ben Hayflick (director); Patrick Muldoon, Bo Svenson"

## 4. Cleaning Processing. Step 2:

To ensure data consistency, we will check for any duplicates in the `original_title` and `title fields`. In case of duplicates, we will manually verify which one is correct based on their respective IDs.

    `28681,"[12, 27]",Equinox,Equinox,"Jack Woods (director/screenplay);` (correct)

    `389035,"[18, 878]",Equinozio,Equinox,"Jack Woods (director/screenplay);` (incorrect)

- 

In [52]:
# Create 'status' column to False
df_director = df[df.duplicated(subset='cast_and_crew', keep=False)]
df_director['status'] = False
df_director

/tmp/ipykernel_2640/3619672055.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_director['status'] = False


,id,genre_ids,original_title,title,cast_and_crew,year,poster_path,status
36,33157.0,"[36, 18, 10752]",Waterloo,Waterloo,Sergei Bondarchuk (director/screenplay); H.A.L...,1970,/tNTLZBn5szEvBRTtkWtAyuFqkDa.jpg,False
37,288959.0,[18],Waterloo,Waterloo,Sergei Bondarchuk (director/screenplay); H.A.L...,1970,NaN,False
47,84496.0,"[80, 18, 53]",Zig Zag,Zig Zag,Richard A. Colla (director); John T. Kelley (s...,1970,/4TvNIjunjz986qDihAlDu33GISV.jpg,False
48,359844.0,[],Zig Zag – Remember the Rules,Zig Zag,Richard A. Colla (director); John T. Kelley (s...,1970,NaN,False
52,28681.0,"[12, 27]",Equinox,Equinox,Jack Woods (director/screenplay); Edward Conne...,1970,/4TbevVaTQ8Vyyez8eEXnoEyDvvq.jpg,False
...,...,...,...,...,...,...,...,...
16986,365620.0,"[18, 36]",Ferrari,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,/nNMoJMDCeF4Q5wpWvKuh5b8K2sX.jpg,False
16987,959286.0,[16],Бизнес войны. Феррари против Ламборгини,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,NaN,False
16988,1036619.0,"[35, 18, 10749]",Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/wJFHUglmh7xlY7g9Ql85QAJBZ6n.jpg,False
16989,1065505.0,[18],Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/4o5qpndKjkLTPHqBrmg6v0W3m13.jpg,False


Since we have duplicates in its `original title` field, let's find the director from API and compare to the `cast_and_crew` colum.

In [49]:
from fuzzywuzzy import fuzz

def is_duplicate(index, row, data_json):
    matched_id = []
    crew = data_json.get('crew')

    for person in crew:
        job = person.get('job', '')
        if job == 'Director':
            name = person.get('name')
            similarity_score = fuzz.partial_ratio(name.lower(), row.cast_and_crew.lower())

            status = similarity_score >= 80

            matched_id.append({
                'index': index,
                'id': int(row.get('id')),
                'original_title': row.get('original_title'),
                'name': name,
                'crew': row.cast_and_crew.split(';')[0],
                'status': status,
            })

    return pd.DataFrame(matched_id)

In [ ]:
load_dotenv()
my_api_key = os.environ.get('tmdb_api_key')

header_df = pd.DataFrame(columns=['index', 'id', 'original_title', 'name', 'crew', 'status'])
FILE_NAME = 'duplicates_.csv'
header_df.to_csv(FILE_NAME, index=False)

i=0
for index, row in tqdm(df_duplicated.iterrows(), total=len(df_duplicated), desc="Processing"):

    movie_id = int(row['id'])
    title = row.title

    while True:
        try:
            response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={my_api_key}")
            response.raise_for_status()

            if response.status_code == 200:
                data_json = response.json()

                result = is_duplicate(index, row, data_json)
                result.to_csv(FILE_NAME, mode='a', header=False, index=False)
                break
            
        except requests.exceptions.RequestException as e:
            print(f"Error for {title}: {e}")
        
        if response.status_code == 429:
            print("Waiting for 10 seconds before retrying...")
            time.sleep(10)
        else:
            break

    time.sleep(0.3)


In [53]:
df_duplicate = pd.read_csv('./data/duplicates.csv', index_col=0)
df_duplicate

,id,original_title,name,crew,status
36,33157,Waterloo,Sergey Bondarchuk,Sergei Bondarchuk (director/screenplay),True
37,288959,Waterloo,Jiří Weiss,Sergei Bondarchuk (director/screenplay),False
47,84496,Zig Zag,Richard A. Colla,Richard A. Colla (director),True
52,28681,Equinox,Dennis Muren,Jack Woods (director/screenplay),False
52,28681,Equinox,Jack Woods,Jack Woods (director/screenplay),True
...,...,...,...,...,...
16987,959286,Бизнес войны. Феррари против Ламборгини,Igor Chernikevich,"Michael Mann (director), Troy Kennedy Martin (...",False
16988,1036619,Good Grief,Dan Levy,Dan Levy (director/screenplay),True
16989,1065505,Good Grief,Rob Sharp,Dan Levy (director/screenplay),False
16990,1011111,Good Grief?,Tim Neeves,Dan Levy (director/screenplay),False


In [54]:
# Identify rows to keep
keep_by_index = df_duplicate.loc[df_duplicate['status'] == True].index

# Update 'status'
df_director.loc[df_director.index.isin(keep_by_index), 'status'] = True
df_director

,id,genre_ids,original_title,title,cast_and_crew,year,poster_path,status
36,33157.0,"[36, 18, 10752]",Waterloo,Waterloo,Sergei Bondarchuk (director/screenplay); H.A.L...,1970,/tNTLZBn5szEvBRTtkWtAyuFqkDa.jpg,True
37,288959.0,[18],Waterloo,Waterloo,Sergei Bondarchuk (director/screenplay); H.A.L...,1970,NaN,False
47,84496.0,"[80, 18, 53]",Zig Zag,Zig Zag,Richard A. Colla (director); John T. Kelley (s...,1970,/4TvNIjunjz986qDihAlDu33GISV.jpg,True
48,359844.0,[],Zig Zag – Remember the Rules,Zig Zag,Richard A. Colla (director); John T. Kelley (s...,1970,NaN,False
52,28681.0,"[12, 27]",Equinox,Equinox,Jack Woods (director/screenplay); Edward Conne...,1970,/4TbevVaTQ8Vyyez8eEXnoEyDvvq.jpg,True
...,...,...,...,...,...,...,...,...
16986,365620.0,"[18, 36]",Ferrari,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,/nNMoJMDCeF4Q5wpWvKuh5b8K2sX.jpg,True
16987,959286.0,[16],Бизнес войны. Феррари против Ламборгини,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,NaN,False
16988,1036619.0,"[35, 18, 10749]",Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/wJFHUglmh7xlY7g9Ql85QAJBZ6n.jpg,True
16989,1065505.0,[18],Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/4o5qpndKjkLTPHqBrmg6v0W3m13.jpg,False


In [55]:
df_director.loc[df_director['status'] == False]

,id,genre_ids,original_title,title,cast_and_crew,year,poster_path,status
37,288959.0,[18],Waterloo,Waterloo,Sergei Bondarchuk (director/screenplay); H.A.L...,1970,NaN,False
48,359844.0,[],Zig Zag – Remember the Rules,Zig Zag,Richard A. Colla (director); John T. Kelley (s...,1970,NaN,False
53,389035.0,"[18, 878]",Equinozio,Equinox,Jack Woods (director/screenplay); Edward Conne...,1970,/8J2tjNmVWUwtXEG1TxFvuNlPGJk.jpg,False
59,654222.0,[99],Getting It Straight in Notting Hill Gate,Getting Straight,Richard Rush (director); Robert Kaufman (scree...,1970,/2kuggGej709c5x8gwiOa4yuqMmw.jpg,False
66,395610.0,[80],Ακόμα μια Φορά... Πριν Ξεψυχήσω,One More Time,Jerry Lewis (director); Michael Pertwee (scree...,1970,/zMV4jLoGdUyx3qdf0T9IjhR7bUr.jpg,False
...,...,...,...,...,...,...,...,...
16982,1032760.0,[99],La memoria infinita,Memory,Michel Franco (director/screenplay); Jessica C...,2023,/7iZSeErC7JdOjdRkuMTfgUxvzx9.jpg,False
16984,1218168.0,[99],Oprah & The Color Purple Journey,The Color Purple,"Blitz Bazawule (director), Marcus Gardley (scr...",2023,/Ap07nfF2bmHsMKNAmLRGU56xlmG.jpg,False
16987,959286.0,[16],Бизнес войны. Феррари против Ламборгини,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,NaN,False
16989,1065505.0,[18],Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/4o5qpndKjkLTPHqBrmg6v0W3m13.jpg,False


In [56]:
df[df.duplicated(subset=['title', 'year'],  keep=False)]

,id,genre_ids,original_title,title,cast_and_crew,year,poster_path
36,33157.0,"[36, 18, 10752]",Waterloo,Waterloo,Sergei Bondarchuk (director/screenplay); H.A.L...,1970,/tNTLZBn5szEvBRTtkWtAyuFqkDa.jpg
37,288959.0,[18],Waterloo,Waterloo,Sergei Bondarchuk (director/screenplay); H.A.L...,1970,NaN
47,84496.0,"[80, 18, 53]",Zig Zag,Zig Zag,Richard A. Colla (director); John T. Kelley (s...,1970,/4TvNIjunjz986qDihAlDu33GISV.jpg
48,359844.0,[],Zig Zag – Remember the Rules,Zig Zag,Richard A. Colla (director); John T. Kelley (s...,1970,NaN
52,28681.0,"[12, 27]",Equinox,Equinox,Jack Woods (director/screenplay); Edward Conne...,1970,/4TbevVaTQ8Vyyez8eEXnoEyDvvq.jpg
...,...,...,...,...,...,...,...
16986,365620.0,"[18, 36]",Ferrari,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,/nNMoJMDCeF4Q5wpWvKuh5b8K2sX.jpg
16987,959286.0,[16],Бизнес войны. Феррари против Ламборгини,Ferrari,"Michael Mann (director), Troy Kennedy Martin (...",2023,NaN
16988,1036619.0,"[35, 18, 10749]",Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/wJFHUglmh7xlY7g9Ql85QAJBZ6n.jpg
16989,1065505.0,[18],Good Grief,Good Grief,"Dan Levy (director/screenplay); Dan Levy, Ruth...",2023,/4o5qpndKjkLTPHqBrmg6v0W3m13.jpg


In [57]:
keep_by_index = df_director.loc[df_director['status'] == False].index

# Delete repeated rows
df_cleaned = df[~df.index.isin(keep_by_index)]

In [58]:
df_cleaned = df_cleaned.drop_duplicates(subset=['title', 'year'], keep='first')

In [59]:
df_cleaned = df_cleaned.drop_duplicates(subset='id')

In [60]:
# Ids are integers
df_cleaned['id'] = df_cleaned['id'].astype(int)

In [61]:
df_cleaned.to_csv('movies_cleaned_.csv', index=False)

In [65]:
df_cleaned.isna().sum()

id                 0
genre_ids          0
original_title     0
title              0
cast_and_crew      0
year               0
poster_path       39
dtype: int64

## 5. Retrieve ratings

In the fifth phase of our data-driven project, we delved into the process of retrieving movie ratings. This step involved accessing and incorporating user-generated ratings.

Leveraging the ratings dataset, we aimed to gauge the popularity, viewer preferences, and overall acclaim for each movie within our curated collection. This wealth of rating information not only enhances the depth of our dataset but also forms the foundation for building a recommender system that tailors movie suggestions based on user preferences.

In [62]:
import numpy as np

def get_ratings(row, data_json):
    matched_id = []

    if data_json.get('total_results'):
        for author in data_json.get('results'):
            id = author.get('id')
            author_details = author.get('author_details')
            rating = author_details.get('rating', np.nan)

            matched_id.append({
                "movieId": int(row.get('id', 0)),
                "userId": id,
                "rating": rating,
            })

    else:
        matched_id.append({
                "movieId": int(row.get('id', 0)),
                "userId": '',
                "rating": '',
            })
        
    matched_id_df = pd.DataFrame(matched_id)

    return matched_id_df

In [ ]:
import requests
from tqdm import tqdm
import time

header_df = pd.DataFrame(columns=['movieId', 'userId', 'rating'])

FILE_NAME = './data/ratings_.csv'
header_df.to_csv(FILE_NAME, index=False)

for _, row in tqdm(df_cleaned.iterrows(), total=len(df_cleaned), desc='Processing'):
    
    movieId = int(row.get('id', 0))

    while True:
        try:
            response = requests.get(f"https://api.themoviedb.org/3/movie/{movieId}/reviews?api_key={my_api_key}")
            response.raise_for_status()

            if response.status_code == 200:
                data_json = response.json()

                current_data = get_ratings(row, data_json)

                current_data.to_csv(FILE_NAME, mode='a', header=False, index=False)

                break

        except requests.exceptions.RequestException as e:
            print(f"Error for {int(movieId)}: {e}")
        
        if response.status_code == 429:
            print(f"Rate limited. Waiting for 10 seconds before retrying...")
            time.sleep(10)
        else:
            break
    
    time.sleep(0.3)

In [67]:
df_rating = pd.read_csv('./data/ratings.csv')
df_rating

,movieId,userId,rating
0,42594,614e280a5ab81a008c5e2b4f,6.0
1,42594,64d4b338dd926a01e988299d,5.0
2,280133,NaN,NaN
3,225155,NaN,NaN
4,85255,NaN,NaN
...,...,...,...
24749,365620,6590032041a56166da04744b,6.0
24750,959286,NaN,NaN
24751,1036619,65a7e9014b0c63012fe35fee,6.0
24752,1065505,NaN,NaN


In [ ]:
df_rating.isna().sum()

In [ ]:
df_rating.dropna(inplace=True)

In [ ]:
len(rating_cleaned)

In [78]:
rating_cleaned = rating_cleaned.drop_duplicates()

In [79]:
# Keep index from movies_cleaned.csv
rating_cleaned = rating_cleaned[rating_cleaned['movieId'].isin(df_cleaned['id'])]

## 6. Anonymizing Users

In order to protect user privacy and comply with data protection regulations, we will anonymize user data.

In [ ]:
# Create a mapping between original user IDs and anonymous IDs
unique_users = rating_cleaned['userId'].unique()

In [ ]:
user_id_mapping = {user_id: f"{idx}" for idx, user_id in enumerate(unique_users)}

In [ ]:
rating_cleaned['userId'] = rating_cleaned['userId'].map(user_id_mapping)

In [ ]:
rating_cleaned.to_csv('ratings_cleaned.csv', index=False)

## 7. Genre Names

In this section, we create a mapping from genre IDs to genre names.

In [6]:
df.genre_ids

0              [27, 878]
1            [18, 10749]
2           [12, 35, 18]
3               [18, 28]
4                   [18]
              ...       
11524               [18]
11525               [18]
11526           [18, 36]
11527           [18, 36]
11528    [35, 18, 10749]
Name: genre_ids, Length: 11529, dtype: object

In [23]:
import json

with open('genres.json', 'r') as fp:
    genres_data = json.load(fp)

genres_mapping = {genre.get('id'): genre.get('name') for genre in genres_data.get('genres')}

In [27]:
from ast import literal_eval

df['genre_ids'] = df['genre_ids'].apply(literal_eval)

In [33]:
df['genre'] = df['genre_ids'].apply(lambda ids: ', '.join(genres_mapping[genre_id] for genre_id in ids))

In [37]:
df.drop('genre_ids', axis=1, inplace=True)

In [39]:
df.drop('original_title', axis=1, inplace=True)

In [46]:
df[df['genre']==''] ='unknown'

/tmp/ipykernel_958/3446741941.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'unknown' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[df['genre']==''] ='unknown'


In [47]:
df.to_csv('movies.csv', index=False)